# Agent Long-Term Memory

Long-term memory persists user facts across sessions using stores with semantic search.

**What you'll learn:**
- Store persists data across sessions and threads
- PostgresStore with embeddings enables semantic search
- Tools access store via runtime.store
- Context provides user identification
- Namespaces organize memories hierarchically
- Semantic search finds relevant memories by meaning

In [ ]:
import sys
sys.path.append('../')

import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool, ToolRuntime
from dataclasses import dataclass

In [ ]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

## Setup PostgresStore

In [ ]:
from langgraph.store.postgres import PostgresStore
import psycopg

embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')

def embed(texts: list[str]):
    return embeddings.embed_documents(texts)

pg_conn = psycopg.connect(os.getenv("POSTGRESQL_URL"), autocommit=True, prepare_threshold=0)
store = PostgresStore(pg_conn, index={"embed": embed, "dims": 768})
store.setup()

# Context schema for user identification
@dataclass
class Context:
    user_id: str

## Memory Tools with ToolRuntime

In [ ]:
# Read memory tool
@tool
def get_user_memory(category: str, runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information from long-term memory."""
    store = runtime.store
    user_id = runtime.context.user_id
    item = store.get((user_id, "preferences"), category)
    return str(item.value) if item else "not found"

# Write memory tool
@tool
def save_user_memory(category: str, information: dict, runtime: ToolRuntime[Context]) -> str:
    """Save user information to long-term memory."""
    store = runtime.store
    user_id = runtime.context.user_id
    store.put((user_id, "preferences"), category, information)
    return "saved"

## Agent with Long-Term Memory

In [ ]:
from langgraph.checkpoint.postgres import PostgresSaver

pg_saver = PostgresSaver(pg_conn)
pg_saver.setup()

agent = create_agent(
    model=model,
    tools=[save_user_memory, get_user_memory],
    checkpointer=pg_saver,
    store=store,
    context_schema=Context
)

In [ ]:
# Save user information
config = {"configurable": {"thread_id": "session_1"}}

agent.invoke({
    'messages': [HumanMessage("Save my info: I'm John from NYC, I prefer Python")]
}, config=config, context=Context(user_id="user_123"))

In [ ]:
# Retrieve in new session (different thread)
new_config = {"configurable": {"thread_id": "session_2"}}

response = agent.invoke({
    'messages': [HumanMessage("What programming language do I prefer?")]
}, config=new_config, context=Context(user_id="user_123"))

response['messages'][-1].text

## Semantic Search

In [ ]:
# Direct store access for semantic search
memories = store.search(("user_123", "preferences"), query="programming", limit=3)

for m in memories:
    print(f"{m.key}: {m.value}")

## Store Direct Operations

In [ ]:
# Direct put/get (without agent)
store.put(("user_456", "preferences"), "language", {"primary": "JavaScript", "learning": "Rust"})

item = store.get(("user_456", "preferences"), "language")
print(item.value)

## Memory Comparison

| Type | Storage | Use Case | Persistence |
|------|---------|----------|-------------|
| **Short-term** | Checkpointer | Conversation history | Session |
| **Long-term** | Store | User preferences, facts | Cross-session |

In [ ]:
# Cleanup
pg_conn.close()